<a href="https://colab.research.google.com/github/tsuli/LSC_project/blob/main/Task3_v01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [5]:
# specify the data file name and url
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00240/'
datafile = url + 'UCI%20HAR%20Dataset.zip'

# download the zip file from the web server using curl
!curl $datafile --output UCI_HAR_Dataset.zip

# unzip the file
!unzip -qq UCI_HAR_Dataset.zip

# change the directory name to remove spaces
!mv -f UCI\ HAR\ Dataset UCI_HAR_DATASET

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 58.1M  100 58.1M    0     0  35.0M      0  0:00:01  0:00:01 --:--:-- 35.0M


In [6]:

xx=np.loadtxt('/content/UCI_HAR_DATASET/test/Inertial Signals/body_acc_x_test.txt').astype(np.float32) 
yy=np.loadtxt('/content/UCI_HAR_DATASET/test/Inertial Signals/body_acc_y_test.txt').astype(np.float32) 
zz=np.loadtxt('/content/UCI_HAR_DATASET/test/Inertial Signals/body_acc_z_test.txt').astype(np.float32) 
xxt=np.loadtxt('/content/UCI_HAR_DATASET/test/Inertial Signals/total_acc_x_test.txt').astype(np.float32) 
yyt=np.loadtxt('/content/UCI_HAR_DATASET/test/Inertial Signals/total_acc_y_test.txt').astype(np.float32) 
zzt=np.loadtxt('/content/UCI_HAR_DATASET/test/Inertial Signals/total_acc_z_test.txt').astype(np.float32) 
xxg=np.loadtxt('/content/UCI_HAR_DATASET/test/Inertial Signals/body_gyro_x_test.txt').astype(np.float32) 
yyg=np.loadtxt('/content/UCI_HAR_DATASET/test/Inertial Signals/body_gyro_y_test.txt').astype(np.float32) 
zzg=np.loadtxt('/content/UCI_HAR_DATASET/test/Inertial Signals/body_gyro_z_test.txt').astype(np.float32) 
# concatenate into a single (n,128,9) array
txtest = np.concatenate((xx[:,:,None],yy[:,:,None],zz[:,:,None],
xxt[:,:,None],yyt[:,:,None],zzt[:,:,None],
xxg[:,:,None],yyg[:,:,None],zzg[:,:,None]),axis=2)

In [7]:
                 
trxx=np.loadtxt('/content/UCI_HAR_DATASET/train/Inertial Signals/body_acc_x_train.txt').astype(np.float32) 
tryy=np.loadtxt('/content/UCI_HAR_DATASET/train/Inertial Signals/body_acc_y_train.txt').astype(np.float32) 
trzz=np.loadtxt('/content/UCI_HAR_DATASET/train/Inertial Signals/body_acc_z_train.txt').astype(np.float32) 
trxxt=np.loadtxt('/content/UCI_HAR_DATASET/train/Inertial Signals/total_acc_x_train.txt').astype(np.float32) 
tryyt=np.loadtxt('/content/UCI_HAR_DATASET/train/Inertial Signals/total_acc_y_train.txt').astype(np.float32) 
trzzt=np.loadtxt('/content/UCI_HAR_DATASET/train/Inertial Signals/total_acc_z_train.txt').astype(np.float32) 
trxxg=np.loadtxt('/content/UCI_HAR_DATASET/train/Inertial Signals/body_gyro_x_train.txt').astype(np.float32) 
tryyg=np.loadtxt('/content/UCI_HAR_DATASET/train/Inertial Signals/body_gyro_y_train.txt').astype(np.float32) 
trzzg=np.loadtxt('/content/UCI_HAR_DATASET/train/Inertial Signals/body_gyro_z_train.txt').astype(np.float32) 
# concatenate into a single (n,128,9) array
txtrain = np.concatenate((trxx[:,:,None],tryy[:,:,None],trzz[:,:,None],
trxxt[:,:,None],tryyt[:,:,None],trzzt[:,:,None],
trxxg[:,:,None],tryyg[:,:,None],trzzg[:,:,None]),axis=2)

In [8]:
ytest = np.loadtxt('UCI_HAR_DATASET/test/y_test.txt').astype(np.float32) -1
ytrain = np.loadtxt('UCI_HAR_DATASET/train/y_train.txt').astype(np.float32) -1

In [9]:
txtrain, txvalid, ytrain, yvalid = train_test_split(txtrain, ytrain, test_size=0.2, random_state=42)

ytrain_enc = tf.keras.utils.to_categorical(ytrain)
yvalid_enc = tf.keras.utils.to_categorical(yvalid)
ytest_enc = tf.keras.utils.to_categorical(ytest)
txtrain = txtrain.astype(np.float32)
txvalid = txvalid.astype(np.float32)
txtest = txtest.astype(np.float32)

print(txtrain.shape, ytrain_enc.shape, txvalid.shape, yvalid_enc.shape, txtest.shape, ytest_enc.shape)


(5881, 128, 9) (5881, 6) (1471, 128, 9) (1471, 6) (2947, 128, 9) (2947, 6)


In [10]:
input_layer = tf.keras.Input(shape=(128,9))
output_layer1 = tf.keras.layers.Conv1D(filters=96, kernel_size=(16), padding = 'same')(input_layer)
output_layer2 = tf.keras.layers.BatchNormalization()(output_layer1)
output_layer3 = tf.keras.layers.ReLU()(output_layer2)
output_layer4 = tf.keras.layers.GlobalAveragePooling1D()(output_layer3)
output_layer5 = tf.keras.layers.Dense(units=6,activation=tf.nn.softmax)(output_layer4)

model = tf.keras.Model(inputs=input_layer, outputs=output_layer5)

In [11]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)]

In [12]:
model.fit(txtrain,  ytrain_enc, 
          epochs=500,
          batch_size=256, 
          callbacks=callbacks,
          validation_data=(txvalid, yvalid_enc))

Epoch 1/500
23/23 [==============================] - 4s 116ms/step - loss: 1.5506 - accuracy: 0.3423 - val_loss: 1.7103 - val_accuracy: 0.2468
Epoch 2/500
23/23 [==============================] - 2s 109ms/step - loss: 1.1993 - accuracy: 0.5897 - val_loss: 1.6340 - val_accuracy: 0.5445
Epoch 3/500
23/23 [==============================] - 3s 110ms/step - loss: 1.0138 - accuracy: 0.8028 - val_loss: 1.5680 - val_accuracy: 0.5289
Epoch 4/500
23/23 [==============================] - 3s 112ms/step - loss: 0.8870 - accuracy: 0.8810 - val_loss: 1.5016 - val_accuracy: 0.5500
Epoch 5/500
23/23 [==============================] - 3s 109ms/step - loss: 0.7950 - accuracy: 0.8980 - val_loss: 1.4352 - val_accuracy: 0.6098
Epoch 6/500
23/23 [==============================] - 3s 111ms/step - loss: 0.7268 - accuracy: 0.9073 - val_loss: 1.3659 - val_accuracy: 0.6873
Epoch 7/500
23/23 [==============================] - 2s 108ms/step - loss: 0.6710 - accuracy: 0.9138 - val_loss: 1.2952 - val_accuracy: 0.7356